In [1]:
from jinja2 import FileSystemLoader, Environment
#from openpyxl import Workbook
#from openpyxl import load_workbook
from glom import glom
import numpy as np

import sys,os,re

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sys.path.append("/Users/szagar/ZTS/Dropbox/Business/ats/Code/Model/v3")
from oos_test import OosTest
from strategy import Strategy
from candidate import Candidate
from prototype import Prototype
from prototype_parameter import PrototypeParameter
from entry_filter import EntryFilter
from poi import Poi

from base import Base, Session
dp = {}
desc ={}
InputsVars = ['longShort', 'sessionStart', 'sessionEnd', 
            'tradesPerDay', 'dayOrSwing', 'poiSwitch',
            'natr', 'fract', 
            'filter1switch', 'filter1n1', 'filter1n2',
            'filter2switch', 'filter2n1', 'filter2n2',
             'tSegment', 'slSwitch', 'stopLoss',
             'ptSwitch', 'profitTarget',
            ]

In [2]:
def nextPrototypeId():
    return session.query(Prototype).filter().order_by(Prototype.id).all()

In [3]:
def queryPrototype(id):
    return session.query(Prototype).filter(Prototype.id == id).all()

In [4]:
def queryPrototypeParams(id):
    return session.query(PrototypeParameter).filter(PrototypeParameter.proto_id == id).all()

In [5]:
def queryEntryFilter(id):
    return session.query(EntryFilter).filter(EntryFilter.id == id).one_or_none()

In [6]:
def queryPoi(id):
    return session.query(Poi).filter(Poi.id == id).one_or_none()

In [7]:
def defineInputsVars():
    #pattern = re.compile(r'^\w+,\w+,\w+$')
    inputs = {}
    varz = {}
    pattern = re.compile(r'^.+,.+,.+$')
    for i in InputsVars:
        if i in dp:
            match = re.search(pattern, str(dp[i]))
            if match:
                if i not in inputs:
                    inputs[i] = {}
                inputs[i]['optStr'] = dp[i]
                inputs[i]['baseVal'] = 'tbd' #candData[i]
                inputs[i]['dtype'] = lookupDtypes(i)   #varRef[i]
            else:
                if i not in varz:
                    varz[i] = {}
                varz[i]['baseVal'] = dp[i] #candData[i]
                varz[i]['dtype'] = lookupDtypes(i)   #varRef[i]
                #print(f'{i}({dp[i]}) :Not found')
    return inputs, varz


In [8]:
dtypes = {'natr': 'int',
         'fract': 'double',
         'filter1_n1': 'int',
         'filter1_n2': 'int',
         'filter2_n1': 'int',
         'filter2_n2': 'int',
          'sl_switch': 'int',
          'stop_loss': 'int',
          'pt_switch': 'int',
          'profit_target': 'int',
          'sess_start': 'int',
          'sess_end': 'int',
          'trades_per_day': 'int',
          'day_or_swing': 'int',
          'lsb': 'int',
          
         }
dtypes['natr']
def lookupDtypes(v):
    if v in dtypes:
        return dtypes[v]
    else:
        return 'tbd'

In [9]:
def render_from_template(directory, template_name, **kwargs):
    loader = FileSystemLoader(directory)
    env = Environment(loader=loader)
    template = env.get_template(template_name)
    return template.render(**kwargs)

In [10]:
def formatCommentStrings(setup):
    print(f'symbol={setup["symbol"]}')
    desc['chart_setup'] = f'''Chart Setup:
        symbol = {setup['symbol']}
        1st timeframe = {setup['timeframe_1']} {setup['timeframe_1_unit']}
        Fitness Function = {setup['fitnessFunc']}
        maxDaysBack = {setup['maxDaysBack']}
        session number = {setup['sessionNum']}
        start date = {setup['start_dt']}
        end date = {setup['end_dt']}'''
    desc['prototype_info'] = f'''Prototype Info::
        dataSet = {setup['data_set']}
        dataBlock = {setup['data_block']}    
        '''

In [11]:
def processTemplate(hdr,inputs,varz,setup,logic):
    templateLoader = FileSystemLoader(searchpath="./proto_templates/")
    templateEnv = Environment(loader=templateLoader)
    TEMPLATE_FILE = 'v1'
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render(header=hdr,
                                 chart_setup=desc['chart_setup'],
                                 prototype_info=desc['prototype_info'],
                                 dp_data=dp,
                                 comments=desc,
                                 inputs = inputs,
                                 varz = varz,
                                 poi = "more testing for poi",
                                 filter_1 = logic['filter_1'],
                                 filter_2 = logic['filter_2'],
                                 setup=setup,
                                 logic=logic
                                )
    #print(outputText)
    return(outputText)

In [24]:
def addInputRange(p):
    inp_def = {}
    inp_def['type'] = 'range'
    if p.data_type == 'float':
        st,end,step = [float(x) for x in p.value.split(',')]
    else:
        st,end,step = [int(x) for x in p.value.split(',')]
    inp_def['value'] = [x for x in np.arange(st,end,step)]
    print(f"inp_def['value']={inp_def['value']}")
    return inp_def

In [25]:
def addInputList(name,inp):
    inp_def = {}
    inp_def['type'] = 'distinct'
    inp_def['value'] = inp
    return name,inp_def

In [26]:
def addInputVar(name,inp):
    inp_def = {}
    inp_def['type'] = 'var'
    inp_def['value'] = inp
    return name,inp_def

In [27]:
def warn(str):
    print(str)

In [28]:
def parseInput(param):
    if param.input_type == 'range':
        return param.name, addInputRange(param)
    elif param.input_type == 'list':
        return addInputList(param.name,param.value)
    elif param.input_type == 'var':
        return addInputVar(param.name,param.value)
    else:
        warn(f'input_type({p.input_type}) not coded for, id={p.id}')

In [29]:
def getConfiguration(id):
    proto = queryPrototype(id)[0]
    params = queryPrototypeParams(id)
    print(f'proto.symbol={proto.symbol}')
    setup = {}
    setup['symbol'] = proto.symbol
    setup['data_set'] = proto.data_set
    setup['data_block'] = proto.data_block
    setup['timeframe_1'] = proto.timeframe_1
    setup['timeframe_1_unit'] = proto.timeframe_1_unit
    setup['fitnessFunc'] = proto.fitness_function
    setup['maxDaysBack'] = proto.max_days_back
    setup['sessionNum'] = proto.session_id
    setup['start_dt'] = proto.start_dt
    setup['end_dt'] = proto.end_dt
    
    setup['opt_inputs'] = {}
    for p in params:
        param_name,param_setup = parseInput(p)
        setup['opt_inputs'][param_name] = param_setup

    return setup

In [30]:
def setHeader(setup):
    hdr = f'''
      template: { template }
      job file: { job_file }
    '''
      #poi IDs     : { setup['poiIds'] }
      #filter 1 IDs: { setup['filterIds_1'] }
      #filter 2 IDs: { setup['filterIds_2'] }

    return hdr

In [31]:
def filterLogic(id_list):
    # {'type': 'distinct', 'ids': '28,29,32'}
    logic = []
    if id_list == '' or id_list == None: return logic
    ids = id_list
    if isinstance(id_list,str):
        ids = id_list.split(',')     
    print(f'ids={ids}')
    print(f'len(ids):{len(ids)}')
    if len(ids) == 0: return []
    for id in ids:
        print(f'id={id}')
        q = queryEntryFilter(id)
        try:
            code = q.long_logic
            logic.append(code)
        except AttributeError:
            code = None
        logic.append(code)
    return logic

In [32]:
def poiLogic(id_list):
    print(f'poiLogic({id_list})')
    # {'type': 'distinct', 'ids': '28,29,32'}
    logic = []
    if id_list == '': return logic
    ids = id_list
    if isinstance(id_list, str):
        ids = id_list.split(',')     
    print(f'ids={ids}')
    print(f'len(ids):{len(ids)}')
    if len(ids) == 0: return []
    for id in ids:
        print(f'id={id}')
        q = queryPoi(id)
        code = q.long_logic
        logic.append(code)
    return logic

In [33]:
def generateEL(proto_id):
    logic = {}
    setup = getConfiguration(proto_id)
    hdr = setHeader(setup)
    inputs, varz = defineInputsVars()
    print(setup['opt_inputs'])
    print(setup['opt_inputs']['filter_1'])
    print(f"glom 1: {glom(setup,'opt_inputs.filter_1')}")
    print(f"glom 2: {glom(setup,'opt_inputs.filter_2',default=None)}")
    print(f"glom 3: {glom(setup,'opt_inputs.poi.value',default=None)}")
    logic['poi'] = poiLogic(glom(setup,'opt_inputs.poi.value',default=None))
    logic['filter_1'] = filterLogic(glom(setup,'opt_inputs.filter_1.value',default=None))
    print(f"logic['filter_1']={logic['filter_1']}")
    #logic['filter_1'] = filterLogic('1,2')
    logic['filter_2'] = filterLogic(glom(setup,'opt_inputs.filter_2',default=None))
    formatCommentStrings(setup)
    return processTemplate(hdr,inputs,varz,setup,logic)
    


In [34]:
db = Session()
db_url = os.environ["DB_HEROKU"]

engine = create_engine(db_url)
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
session = DBSession()

## 

In [35]:
proto_id = nextPrototypeId()[0].id
template = 'v1'
job_file = f'opt_job_proto_{proto_id}'
out = generateEL(proto_id)
print(out)

proto.symbol=@RTY.D
inp_def['value']=[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
inp_def['value']=[1, 2, 3, 4, 5, 6, 7]
inp_def['value']=[0.6, 0.75, 0.9, 1.05, 1.2000000000000002, 1.35, 1.5, 1.6500000000000004, 1.8000000000000003, 1.9500000000000002, 2.1, 2.2500000000000004, 2.4000000000000004, 2.5500000000000003, 2.7000000000000006, 2.8500000000000005]
{'natr': {'type': 'range', 'value': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]}, 'poi_n1': {'type': 'var', 'value': '-1'}, 'poi': {'type': 'range', 'value': [1, 2, 3, 4, 5, 6, 7]}, 'fract': {'type': 'range', 'value': [0.6, 0.75, 0.9, 1.05, 1.2000000000000002, 1.35, 1.5, 1.6500000000000004, 1.8000000000000003, 1.9500000000000002, 2.1, 2.2500000000000004, 2.4000000000000004, 2.5500000000000003, 2.7000000000000006, 2.8500000000000005]}, 'filter_1': {'type': 'distinct', 'value': '2,5,6'}}
{'type': 'distinct', 'value': '2,5,6'}
glom 1: {'type': 'distinct', 'value': '2,5,6'}
glom 2: None
glom 3: [1, 2, 3, 4, 5, 6, 7]
poiLogic([1, 2, 3, 4, 5,

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'numpy.int64' [SQL: 'SELECT pois.id AS pois_id, pois.poi_type AS pois_poi_type, pois.long_logic AS pois_long_logic, pois.short_logic AS pois_short_logic, pois.common_logic AS pois_common_logic, pois.long_params AS pois_long_params, pois.short_params AS pois_short_params, pois.vars AS pois_vars, pois.status AS pois_status \nFROM pois \nWHERE pois.id = %(id_1)s'] [parameters: {'id_1': 1}] (Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
import numpy as np
a = [x for x in np.arange(5,60,5)]

In [ ]:
a